In [71]:
#https://www.kaggle.com/tannistha/hyperparameter-grid-search-with-xgboost/edit
#https://github.com/IBM/xgboost-smote-detect-fraud
#https://stats.stackexchange.com/questions/179835/how-to-build-a-confusion-matrix-for-a-multiclass-classifier

In [79]:
#!/usr/bin/python

from __future__ import division

import numpy as np
import xgboost as xgb
import pandas as pd
from itertools import product
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search
from sklearn.model_selection import KFold
from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn import decomposition

In [103]:
data_spectrum =  pd.read_csv('Spectrum_baseline.csv') #'Spectrum_baseline.csv'
labels = pd.read_csv('Spectrum_labels.csv').values
labels_new = pd.DataFrame({'Class':labels[:,1].astype('int64')})
total_df = pd.concat([data_spectrum, labels_new], axis =1)
feature = total_df.columns
data = total_df[feature[1:]].values

In [104]:
# without last column
pca = decomposition.PCA(n_components=200)
pca.fit(data[:,:-1])
data = pca.transform(data[:,:-1])

In [105]:
data_new= np.append(data, total_df.iloc[:,-1].values.reshape(11442,1), axis=1)

In [106]:
sz = data_new.shape
np.random.shuffle(data_new)

In [109]:
train = data_new[:int(sz[0] * 0.7), :]
test = data_new[int(sz[0] * 0.7):, :]

In [110]:
train_X = train[:, :200]
train_Y = train[:, 200]
test_X = test[:, :200]
test_Y = test[:, 200]

In [111]:
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['scale_pos_weight']:[0.33, 0.7, 0.167,0.3,0.4]
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 5

In [112]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(train_X, train_Y)#ratio={1:1000, 3:1000, 4:1000}
X_res, y_res = sm.fit_sample(train_X, train_Y)
xg_train = xgb.DMatrix(X_res, label=y_res)
xg_test = xgb.DMatrix(test_X, label=test_Y)
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5

/home/titli/anaconda3/lib/python3.6/site-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


In [113]:
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
print('Test error using softmax = {}'.format(error_rate))

[0]	train-merror:0.320382	test-merror:0.506263
[1]	train-merror:0.293392	test-merror:0.492572
[2]	train-merror:0.269288	test-merror:0.482668
[3]	train-merror:0.259097	test-merror:0.48092
[4]	train-merror:0.241926	test-merror:0.468395
Test error using softmax = 0.4683949898048354


In [114]:
# do the same thing again, but output probabilities
param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bst.predict(xg_test).reshape(test_Y.shape[0], 5)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != test_Y) / test_Y.shape[0]
print('Test error using softprob = {}'.format(error_rate))

[0]	train-merror:0.320382	test-merror:0.506263
[1]	train-merror:0.293392	test-merror:0.492572
[2]	train-merror:0.269288	test-merror:0.482668
[3]	train-merror:0.259097	test-merror:0.48092
[4]	train-merror:0.241926	test-merror:0.468395
Test error using softprob = 0.4683949898048354


In [115]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(pred, test_Y)
recall = np.diag(cm) / np.sum(cm, axis = 1)
precision = np.diag(cm) / np.sum(cm, axis = 0)
print(recall, precision)

[0.71280277 0.17035775 0.801      0.30493274 0.31894934] [0.87288136 0.47619048 0.42448331 0.42236025 0.55555556]
